In [130]:
# Import Dependencies
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
from sqlalchemy import create_engine
import codecs
import re
import ast
from pprint import pprint

requests.__version__

'2.27.1'

In [131]:
# Import the API key

from config import yelp_api_key


In [132]:
city_file_to_load = "Resources/zip_code_database.csv"
# Open the city_file to filter zip codes for metropolitan areas.
city_file_df = pd.read_csv(city_file_to_load)

In [133]:
cities_df = city_file_df[(city_file_df.zip >= 10001) & 
                         (city_file_df.irs_estimated_population >= 50000) &
                        (city_file_df.state.isin(["WA", "IL", "NY"]))]

cities_df.head(10)

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
3762,10002,STANDARD,0,New York,Knickerbocker,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,718,NaN,US,40.71,-73.99,63960
3783,10023,STANDARD,0,New York,NaN,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,"212,347,646,718,917",NaN,US,40.78,-73.98,50320
3785,10025,STANDARD,0,New York,NaN,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,"212,646,917",NaN,US,40.80,-73.97,67440
3789,10029,STANDARD,0,New York,NaN,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,"212,646,917",NaN,US,40.79,-73.94,57070
3931,10306,STANDARD,0,Staten Island,NaN,NaN,NY,Richmond County,America/New_York,718,NaN,US,40.58,-74.14,52070
3937,10312,STANDARD,0,Staten Island,NaN,NaN,NY,Richmond County,America/New_York,"347,917,929,718",NaN,US,40.55,-74.18,56690
3939,10314,STANDARD,0,Staten Island,NaN,NaN,NY,Richmond County,America/New_York,"347,718,917,929",NaN,US,40.60,-74.17,80070
3941,10452,STANDARD,0,Bronx,NaN,"Highbridge, Stadium, University Heights",NY,Bronx County,America/New_York,"347,646,917,929,212,718",NaN,US,40.84,-73.92,68040
3942,10453,STANDARD,0,Bronx,NaN,NaN,NY,Bronx County,America/New_York,718,NaN,US,40.85,-73.91,70760
3945,10456,STANDARD,0,Bronx,NaN,Morrisania,NY,Bronx County,America/New_York,"718,212,347,646,917,929",NaN,US,40.83,-73.91,77400


In [161]:
# convert cities_df to a list to loop through a list of zip codes for the api
cities = cities_df["zip"].tolist()
# cities = 10002

In [7]:
# count of zip codes
#cities = [10002]

99

In [168]:
# Starting URL for Yelp API call.
# url = "https://api.yelp.com/v3/businesses/search?location="
data_file = open('data_file.json', 'w', encoding="utf-8")

#csv_writer = csv.writer(data_file)

restaurants = {"data":[]}

for city in cities:
    
    # headers contain the api key.
    headers = {'Authorization': 'Bearer {}'.format(yelp_api_key)}
    search_api_url = 'https://api.yelp.com/v3/businesses/search'
    params = {'term': 'Restaurants', 
              'location': city,
              'limit': 50}
    
    response = requests.get(search_api_url, headers=headers, params=params, timeout=5)
    
    data_dict = response.json()
    if "businesses" in data_dict:
            
        for restaurant in data_dict["businesses"]:
            #print(restaurant.values())
            #csv_writer.writerow(restaurant.values())
            
          #  fifty_set.append(restaurant)
            
            restaurants["data"].append(restaurant)

json.dump(restaurants, data_file, indent=4, sort_keys=True)
data_file.close()


In [146]:
import os
print(os.getcwd())

C:\Users\Junlee\Desktop\DataClass\Yelp_Metro_US_Analysis_2\Yelp_Metro_US_Analysis


In [169]:
# Create file directory variable (update directory)
file_dir = 'C:/Users/Junlee/Desktop/DataClass/Yelp_Metro_US_Analysis_2/Yelp_Metro_US_Analysis/'

# load json file to read
with open (f'{file_dir}data_file.json',mode = 'r', encoding="utf-8") as file:
    #yelp_raw_data = pd.read_json(file, lines=True)
    #yelp_raw_data=ast.literal_eval(json.load(file))
    #pd.DataFrame(
    #yelp_raw_data=pd.read_json(codecs.open('C:/Users/Junlee/Desktop/DataClass/Yelp_Metro_US_Analysis/data_file.json','r','utf-8'), lines=True, orient='records')
    yelp_raw_data=json.load(file)
    yelp_panda_data=pd.DataFrame(yelp_raw_data)
yelp_panda_data.head(20)  
   

,data
0,"{'alias': 'yes-apothecary-new-york-4', 'catego..."
1,"{'alias': 'la-contenta-new-york', 'categories'..."
2,"{'alias': 'the-cabin-nyc-new-york-2', 'categor..."
3,"{'alias': '8282-new-york-2', 'categories': [{'..."
4,"{'alias': 'joes-shanghai-new-york-2', 'categor..."
5,"{'alias': 'mắm-new-york-2', 'categories': [{'a..."
6,"{'alias': 'betty-new-york', 'categories': [{'a..."
7,"{'alias': '3-times-new-york-5', 'categories': ..."
8,"{'alias': 'wayla-new-york', 'categories': [{'a..."
9,"{'alias': 'sweet-chick-new-york', 'categories'..."


In [170]:
len(yelp_panda_data)

4950

In [171]:
yelp_raw_data

{'data': [{'alias': 'yes-apothecary-new-york-4',
   'categories': [{'alias': 'szechuan', 'title': 'Szechuan'},
    {'alias': 'speakeasies', 'title': 'Speakeasies'}],
   'coordinates': {'latitude': 40.71945, 'longitude': -73.9898},
   'display_phone': '',
   'distance': 479.59624982230207,
   'id': '8YWLuLUKj0t_0_Xv06UUtw',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/C9kLckBI7NTHBnHV-98qEw/o.jpg',
   'is_closed': False,
   'location': {'address1': '119 Orchard St',
    'address2': None,
    'address3': '',
    'city': 'New York',
    'country': 'US',
    'display_address': ['119 Orchard St', 'New York, NY 10002'],
    'state': 'NY',
    'zip_code': '10002'},
   'name': "Ye's Apothecary",
   'phone': '',
   'rating': 4.5,
   'review_count': 105,
   'transactions': [],
   'url': 'https://www.yelp.com/biz/yes-apothecary-new-york-4?adjust_creative=n-qoFaVTItShXPgt7XBAuQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=n-qoFaVTItShXPgt7XBAuQ'},
  {'alias': '

In [172]:
# yelp_panda_data=pd.DataFrame(yelp_raw_data)
# yelp_panda_data.head(20)
# print(yelp_raw_data["data"][1])

places =[]
yelp_raw_data_2 = yelp_raw_data
yelp_panda_data_by_zip = []
# print(yelp_raw_data["data"])
yelp_panda_data_by_zip=pd.DataFrame(yelp_raw_data["data"])
yelp_panda_data_by_zip.head(20)

# for panda_item in yelp_raw_data_2['data'][0].values():
#     yelp_panda_data_by_zip=pd.DataFrame(panda_item)
# #     print(panda_item)
#     yelp_panda_data_by_zip.head(20)
# for city_index in range(1):
#     for panda_item in yelp_raw_data_2['data'][city_index].values():
# #         print("zip code ", city_index,"***************")
# #         print(panda_item)
#         yelp_panda_data_by_zip=pd.DataFrame(panda_item)
#         print(panda_item)
#         yelp_panda_data_by_zip.head(20)
        

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,rating,review_count,transactions,url,price
0,yes-apothecary-new-york-4,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...","{'latitude': 40.71945, 'longitude': -73.9898}",,479.596250,8YWLuLUKj0t_0_Xv06UUtw,https://s3-media1.fl.yelpcdn.com/bphoto/C9kLck...,False,"{'address1': '119 Orchard St', 'address2': Non...",Ye's Apothecary,,4.5,105,[],https://www.yelp.com/biz/yes-apothecary-new-yo...,NaN
1,la-contenta-new-york,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.71873, 'longitude': -73.98691}",(212) 432-4180,274.155979,6Ct7OeTRn4caY9g0swFBsw,https://s3-media2.fl.yelpcdn.com/bphoto/FwZAyP...,False,"{'address1': '102 Norfolk St', 'address2': '',...",La Contenta,+12124324180,4.5,865,"[pickup, delivery]",https://www.yelp.com/biz/la-contenta-new-york?...,$$
2,the-cabin-nyc-new-york-2,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.72393, 'longitude': -73.98383}",(212) 777-0454,829.052905,J3NT61-AH5d5Gu5tFJhYSw,https://s3-media4.fl.yelpcdn.com/bphoto/fbKZ7H...,False,"{'address1': '205 E 4th St', 'address2': None,...",The Cabin NYC,+12127770454,4.0,495,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/the-cabin-nyc-new-yor...,$$
3,8282-new-york-2,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 40.72161, 'longitude': -73.98875}",(929) 837-0360,623.059893,fNKYBpQJivICm0ORb3nb_Q,https://s3-media2.fl.yelpcdn.com/bphoto/jMQtrS...,False,"{'address1': '84 Stanton St', 'address2': None...",8282,+19298370360,4.5,105,[],https://www.yelp.com/biz/8282-new-york-2?adjus...,NaN
4,joes-shanghai-new-york-2,"[{'alias': 'shanghainese', 'title': 'Shanghain...","{'latitude': 40.7156608, 'longitude': -73.9967...",(212) 233-8888,946.852614,0CjK3esfpFcxIopebzjFxA,https://s3-media4.fl.yelpcdn.com/bphoto/xM4eGR...,False,"{'address1': '46 Bowery St', 'address2': '', '...",Joe's Shanghai,+12122338888,4.0,6937,"[pickup, delivery]",https://www.yelp.com/biz/joes-shanghai-new-yor...,$$
5,mắm-new-york-2,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]","{'latitude': 40.71704, 'longitude': -73.99338}",,653.825448,Bn25orKr4DP_Onk3iS-QZw,https://s3-media1.fl.yelpcdn.com/bphoto/E8dvBe...,False,"{'address1': '70 Forsyth St', 'address2': '', ...",Mắm,,4.5,20,[],https://www.yelp.com/biz/m%E1%BA%AFm-new-york-...,NaN
6,betty-new-york,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.71367, 'longitude': -73.98785}",(646) 556-5081,372.954463,c8wkoGg9UbF55k3Z74X0Qw,https://s3-media1.fl.yelpcdn.com/bphoto/cyj4EA...,False,"{'address1': '193 Henry St', 'address2': None,...",Betty,+16465565081,5.0,7,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/betty-new-york?adjust...,NaN
7,3-times-new-york-5,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...","{'latitude': 40.718271, 'longitude': -73.985209}",(646) 609-6324,183.445542,xLJnT-pB4VECJzmUf1Whpg,https://s3-media2.fl.yelpcdn.com/bphoto/uBC9JN...,False,"{'address1': '90 Clinton St', 'address2': '', ...",3 Times,+16466096324,4.0,206,"[pickup, restaurant_reservation, delivery]",https://www.yelp.com/biz/3-times-new-york-5?ad...,$$
8,wayla-new-york,"[{'alias': 'thai', 'title': 'Thai'}]","{'latitude': 40.7183, 'longitude': -73.9926038}",(212) 206-2500,623.989139,liZmIO0FqZwslLJfLDkfgw,https://s3-media2.fl.yelpcdn.com/bphoto/u17PWW...,False,"{'address1': '100 Forsyth St', 'address2': 'Ba...",Wayla,+12122062500,4.5,545,[delivery],https://www.yelp.com/biz/wayla-new-york?adjust...,$$$
9,sweet-chick-new-york,"[{'alias': 'southern', 'title': 'Southern'}, {...","{'latitude': 40.7217399, 'longitude': -73.98739}",(646) 657-0233,582.427178,8F81LrdFm6P5kkFpsw0n0w,https://s3-media4.fl.yelpcdn.com/bphoto/e7M1L-...,False,"{'address1': '178 Ludlow St', 'address2': '', ...",Sweet Chick,+16466570233,4.0,1849,"[pickup, delivery]",https://www.yelp.com/biz/sweet-chick-new-york?...,$$


In [174]:
len(yelp_panda_data_by_zip)

4950

In [176]:
yelp_panda_data_by_zip.to_csv("yelp_data.csv", encoding='utf-8', index=False)